# Webscrape college applications
## Step One: Prepare Directory Information

In [1]:
# import sys
# log_file = open('log_file.txt', 'w')
# sys.stdout = log_file
# print('End of cell')

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
# Load directory data.
IPEDSfile = pd.read_stata('../../statadata/IPEDSDirInfo02to16smlr.dta', preserve_dtypes=False)
# Reduce to 2 and 4yr institutions.
IPEDSfile['filter'] = np.where((IPEDSfile['sector']=='Public, 4-year or above') | 
                               (IPEDSfile['sector']=='Public, 2-year') |
                               (IPEDSfile['sector']=='Private not-for-profit, 4-year or above') |
                               (IPEDSfile['sector']=='Private not-for-profit, 2-year'), 1, 0)
IPEDSfile = IPEDSfile[IPEDSfile['filter']==1]

In [4]:
# Remove www. prefix from webaddress.
IPEDSfile['rootdom'] = IPEDSfile.loc[:, 'webaddr'].replace(regex=True, to_replace='www.', value='')
# Remove miscellaneous slashes & other from webaddress.
IPEDSfile['rootdom'] = IPEDSfile.loc[:, 'rootdom'].replace(regex=True, to_replace=r'/', value='')
IPEDSfile['rootdom'] = IPEDSfile.loc[:, 'rootdom'].replace(regex=True, to_replace=r'HTTPS:', value='')

In [5]:
IPEDSfile[['unitid','instnm','rootdom','filter','sector','iclevel']].head(n=15)

,unitid,instnm,rootdom,filter,sector,iclevel
0,100654,Alabama A & M University,aamu.edu,1,"Public, 4-year or above",Four or more years
1,100663,University of Alabama at Birmingham,uab.edu,1,"Public, 4-year or above",Four or more years
2,100690,Amridge University,amridgeuniversity.edu,1,"Private not-for-profit, 4-year or above",Four or more years
3,100706,University of Alabama in Huntsville,uah.edu,1,"Public, 4-year or above",Four or more years
4,100724,Alabama State University,alasu.edu,1,"Public, 4-year or above",Four or more years
6,100751,The University of Alabama,ua.edu,1,"Public, 4-year or above",Four or more years
7,100760,Central Alabama Community College,cacc.edu,1,"Public, 2-year",At least 2 but less than 4 years
8,100812,Athens State University,athens.edu,1,"Public, 4-year or above",Four or more years
9,100830,Auburn University at Montgomery,aum.edu,1,"Public, 4-year or above",Four or more years
10,100858,Auburn University,auburn.edu,1,"Public, 4-year or above",Four or more years


In [6]:
IPEDSfile.describe()

,unitid,isYr,fte,enrtot,filter
count,3645.000000,3645.0,0.0,0.0,3645.0
mean,215559.690261,2016.0,NaN,NaN,1.0
std,98226.949385,0.0,NaN,NaN,0.0
min,100654.000000,2016.0,NaN,NaN,1.0
25%,154101.000000,2016.0,NaN,NaN,1.0
50%,194116.000000,2016.0,NaN,NaN,1.0
75%,227854.000000,2016.0,NaN,NaN,1.0
max,489937.000000,2016.0,NaN,NaN,1.0


In [7]:
# Create list of domains to scrape
weblist = IPEDSfile['rootdom']

## Step Two: Prepare Browser Object

In [8]:
import webbrowser
from time import sleep
from selenium import webdriver
import os
import requests
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [9]:
browser = webdriver.Firefox()
print('Loaded Browser Here')
# browser = webdriver.Chrome()

Loaded Browser Here


In [10]:
# browser.get(r'https://www.google.com/search?as_q=early+decision&as_epq=&as_sitesearch=.edu&as_filetype=pdf')
# print('send browser to search')

# Step Three: Scrape For Each School

In [28]:
error_list = []
for school in weblist:
    print('STARTING SCHOOL', str(school).upper())
    searchstr = ''.join((r'https://www.google.com/search?as_q=early+decision&as_epq=&as_sitesearch=', 
                        school, '&as_filetype=pdf'))
    browser.get(searchstr)
    print('Search string is ', searchstr)
    results = browser.find_elements_by_css_selector('h3 > a')
    if len(results) == 0:
        print('No results for school ', school.lower(), end='\n\n')
    for i in range (len(results)):
    # for i in range(3):
        filelink = results[i].get_attribute('href')
        try:
            pdf = requests.get(filelink)
        except ConnectionError:
            print('There was a ConnectionError on the {}th iteration at : {}'.format(str(i), school))
            error_list.append(''.join(school, str(i), 'ConnectionError')
        except:
            print('There was an UnspecifiedError on the {}th iteration at : {}'.format(str(i), school))
            error_list.append(''.join(school, str(i), 'UnspecifiedError')
        fname = ''.join((school,str(i),'.pdf'))
        print('Make filename.')
        print('Filelink = ' + filelink)
        print('Saved as : ' + fname)
        print('Header info :' + str(pdf.headers), end='\n\n')
        open(''.join((r'pdfs/', fname)), 'wb').write(pdf.content)

# log_file.close()

SyntaxError: invalid syntax (<ipython-input-28-234e943fb20d>, line 19)